In [2]:
import os
import numpy as np
import pprint
import matplotlib.pyplot as plt
import wave

def get_raw_data(filename):
    with wave.open(filename, 'rb') as wav_file:
        # ヘッダー情報を取得
        n_channels = wav_file.getnchannels()
        sample_width = wav_file.getsampwidth()
        frame_rate = wav_file.getframerate()
        n_frames = wav_file.getnframes()

        print(f'チャンネル数: {n_channels}')
        print(f'サンプル幅 (バイト): {sample_width}')
        print(f'フレームレート (Hz): {frame_rate}')
        print(f'フレーム数: {n_frames}')
        print(f'ビット深度: {sample_width * 8}')  # ビット深度はサンプル幅に8を掛けたもの

        # WAVファイルのヘッダーを除いたデータ部分を取得
        raw_data = wav_file.readframes(n_frames)
        return raw_data

# audacityでRAW(header-ledd) signed 16bit pcmで出したデータを
# ノーマライズののちsigned 11bitの波形でC headerファイルとして出力
def convert_wavefile_to_c_array(filename, out_path):

    raw_data = get_raw_data(filename)
    # 符号16ビットRAW波形ファイルを読み込む
    data = np.frombuffer(raw_data, dtype=np.int16)
    # data = np.fromfile(filename, dtype=np.int16)

    # データ長が1024の倍数になるように0を追加
    # if len(data) % 1024 != 0:
    #     padding_length = 1024 - (len(data) % 1024)
    #     data = np.concatenate((data, np.zeros(padding_length, dtype=np.int16)))

    # ノーマライズ→11bit(符号ありなのでシフト数は10bit)
    data = data / np.max(np.abs(data))
    data = (data * ((1 << 10) - 1)).astype(np.int16)

    # plt.plot(data)
    # plt.show()

    # 16>11
    # data = data >> 5

    # 16個ずつ改行するのに変換
    c_array = []
    for i in range(0, len(data), 16):
        c_array.append(",".join(map(str, data[i:i+16])))

    wave_length = len(data)

    # 配列変数定義ファイルを作成
    filename = os.path.basename(filename)
    filename_without_extension = os.path.splitext(filename)[0]
    val_len = f'buf_size_{filename_without_extension}'
    val_name = f'buf_{filename_without_extension}'
    txt = '#pragma once\n'
    txt += '#include <Arduino.h>\n'
    txt += f'const uint16_t {val_len} = {wave_length}; \n'
    txt += f'const int16_t {val_name}[{wave_length}] = ' + '{\n'
    txt += ',\n'.join(c_array)
    txt += '};\n'
    header_name = f"{filename_without_extension}.h"
    header_path = os.path.join(out_path, header_name)
    with open(header_path, "w") as f:
        f.write(txt)
    return header_path, val_len, val_name


import os
import glob
import wave

in_path = 'wavetable/808_01_wav/'
out_path = 'wavetable/808_01/'

if not os.path.exists(out_path):
    os.makedirs(out_path)

# フォルダ内のすべてのWAVファイルを取得
wav_files = glob.glob(os.path.join(in_path, '*.wav'))

includes = ''
classobjects = ''
index = 0
for wav_file_path in wav_files:
    filename = os.path.basename(wav_file_path)
    print(wav_file_path)
    header_name, val_len, val_name = convert_wavefile_to_c_array(wav_file_path, out_path)
    includes += f'#include "{header_name}"\n'
    classobjects += f'SingleShotWave ssw{index}({val_name}, {val_len});\n'
    index += 1


print('------------------inlude list')
print(includes)
print(classobjects)



wavetable/808_01_wav\808_BD_02.wav
チャンネル数: 1
サンプル幅 (バイト): 2
フレームレート (Hz): 44100
フレーム数: 30848
ビット深度: 16
wavetable/808_01_wav\808_OHH_01.wav
チャンネル数: 1
サンプル幅 (バイト): 2
フレームレート (Hz): 44100
フレーム数: 16835
ビット深度: 16
wavetable/808_01_wav\808_Rim_01.wav
チャンネル数: 1
サンプル幅 (バイト): 2
フレームレート (Hz): 44100
フレーム数: 607
ビット深度: 16
wavetable/808_01_wav\808_SD_01.wav
チャンネル数: 1
サンプル幅 (バイト): 2
フレームレート (Hz): 44100
フレーム数: 6165
ビット深度: 16
------------------inlude list
#include "wavetable/808_01/808_BD_02.h"
#include "wavetable/808_01/808_OHH_01.h"
#include "wavetable/808_01/808_Rim_01.h"
#include "wavetable/808_01/808_SD_01.h"

SingleShotWave ssw0(buf_808_BD_02, buf_size_808_BD_02);
SingleShotWave ssw1(buf_808_OHH_01, buf_size_808_OHH_01);
SingleShotWave ssw2(buf_808_Rim_01, buf_size_808_Rim_01);
SingleShotWave ssw3(buf_808_SD_01, buf_size_808_SD_01);

